In [8]:
import random
import numpy as np
import mnist_loader

# Fully Connected Neural Network for MNIST classification
This notebook provides a code example of how to use a Deep Fully Connected Neural Network for MNIST classification. The MNIST task is the "hello world" of computer vision - the task asks a classifier to label an image of a digit as the digit it represents.

Code adapted from [Michael Nielsen's](http://michaelnielsen.org/) code for his book http://neuralnetworksanddeeplearning.com/.

### Data Loading

In [9]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

### Neural Network Class

In [10]:
class DeepFeedforwardNetwork(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    
    # Here mini batch size refers to the number of observations in each mini-batch, epochs refers to 
    # how many times we use the training data in getting collections of mini-batches, and eta refers
    # to the learning rate. 
    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for epoch in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Test accuracy on epoch {}: {}".format(epoch, self.evaluate(test_data) / n_test))
            else:
                print("Epoch " + str(epoch) + "complete")

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

Here we initialize a Deep Feedforward Network with 784 inputs (784 neurons in the input layer), 30 neurons in the hidden layer, and 10 neurons in the output layer. 

In [11]:
# initialize the network
net = DeepFeedforwardNetwork([784, 30, 10])

In [12]:
num_epochs = 30
batch_size = 10
learning_rate = 3.0 
net.SGD(training_data, epochs=num_epochs, mini_batch_size=batch_size, eta=learning_rate, test_data=test_data)

Test accuracy on epoch 0: 0.9039
Test accuracy on epoch 1: 0.9238
Test accuracy on epoch 2: 0.9309
Test accuracy on epoch 3: 0.9322
Test accuracy on epoch 4: 0.9326
Test accuracy on epoch 5: 0.9389
Test accuracy on epoch 6: 0.9378
Test accuracy on epoch 7: 0.9407
Test accuracy on epoch 8: 0.9393
Test accuracy on epoch 9: 0.9454
Test accuracy on epoch 10: 0.9455
Test accuracy on epoch 11: 0.9443
Test accuracy on epoch 12: 0.9429
Test accuracy on epoch 13: 0.9461
Test accuracy on epoch 14: 0.9459
Test accuracy on epoch 15: 0.9445
Test accuracy on epoch 16: 0.9483
Test accuracy on epoch 17: 0.9477
Test accuracy on epoch 18: 0.9481
Test accuracy on epoch 19: 0.9491
Test accuracy on epoch 20: 0.9432
Test accuracy on epoch 21: 0.9498
Test accuracy on epoch 22: 0.951
Test accuracy on epoch 23: 0.9493
Test accuracy on epoch 24: 0.9491
Test accuracy on epoch 25: 0.95
Test accuracy on epoch 26: 0.9502
Test accuracy on epoch 27: 0.9502
Test accuracy on epoch 28: 0.9476
Test accuracy on epoch 29: 